<a href="https://colab.research.google.com/github/123Vind/Diabetis-Predictor/blob/master/diabetisNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [365]:
!pip install imbalanced-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [366]:
import tensorflow as tf
import numpy as np
import pandas as pd

from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import TomekLinks


In [367]:
#importing dataset from google drive
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [368]:
diabetis =  pd.read_csv('/content/drive/My Drive/Colab Notebooks/diabetis.csv',header = None)

In [369]:
diabetis.head

<bound method NDFrame.head of       0    1   2   3    4     5      6   7  8
0     6  148  72  35    0  33.6  0.627  50  1
1     1   85  66  29    0  26.6  0.351  31  0
2     8  183  64   0    0  23.3  0.672  32  1
3     1   89  66  23   94  28.1  0.167  21  0
4     0  137  40  35  168  43.1  2.288  33  1
..   ..  ...  ..  ..  ...   ...    ...  .. ..
763  10  101  76  48  180  32.9  0.171  63  0
764   2  122  70  27    0  36.8  0.340  27  0
765   5  121  72  23  112  26.2  0.245  30  0
766   1  126  60   0    0  30.1  0.349  47  1
767   1   93  70  31    0  30.4  0.315  23  0

[768 rows x 9 columns]>

In [370]:
X = diabetis.iloc[:,:-1]
y = diabetis.iloc[:,-1]


In [371]:
#scaling the training data
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)


In [372]:
X

,0,1,2,3,4,5,6,7
0,6,148,72,35,0,33.6,0.627,50
1,1,85,66,29,0,26.6,0.351,31
2,8,183,64,0,0,23.3,0.672,32
3,1,89,66,23,94,28.1,0.167,21
4,0,137,40,35,168,43.1,2.288,33
...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63
764,2,122,70,27,0,36.8,0.340,27
765,5,121,72,23,112,26.2,0.245,30
766,1,126,60,0,0,30.1,0.349,47


In [373]:

y.value_counts()


0    500
1    268
Name: 8, dtype: int64

In [374]:
#to get rid of the unbalanced class
oversample = SMOTE()
X_res, y_res = oversample.fit_resample(X_scaled, y)

In [375]:
X_train, X_test, y_train, y_test = train_test_split(X_res,y_res,random_state = 2,test_size = 0.20,stratify = y_res)

In [376]:
X_train

array([[0.23529412, 0.75879397, 0.73770492, ..., 0.44262295, 0.09222886,
        0.25      ],
       [0.17647059, 0.81909548, 0.57377049, ..., 0.4709389 , 0.08112724,
        0.11666667],
       [0.23529412, 0.72864322, 0.67213115, ..., 0.48435171, 0.06703672,
        0.81666667],
       ...,
       [0.11764706, 0.50753769, 0.47540984, ..., 0.32488823, 0.03287788,
        0.01666667],
       [0.29411765, 0.49748744, 0.60655738, ..., 0.43219076, 0.05337319,
        0.18333333],
       [0.41176471, 0.92462312, 0.68852459, ..., 0.5290611 , 0.11827498,
        0.33333333]])

In [377]:
y_res.value_counts()

1    500
0    500
Name: 8, dtype: int64

In [378]:
y_train.shape

(800,)

In [379]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
num_classes = 2

In [380]:
X_train.shape

(800, 8)

In [381]:
def build_model():
  model = Sequential()
  model.add(Dense(6,input_dim=8,activation='relu'))
  model.add(Dense(8,activation='relu'))
  model.add(Dense(10,activation='relu'))
  model.add(Dense(8,activation='relu'))
  model.add(Dense(num_classes,activation='sigmoid'))
  model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
  return model

In [382]:
model = build_model()

In [383]:
y_train.shape

(800, 2)

In [384]:
y_train

array([[1., 0.],
       [0., 1.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [0., 1.]], dtype=float32)

In [385]:
model.summary()
sample_weight = np.ones(shape=(len(y_train),2))
#sample_weight.shape
sample_weight[y_train==[0.,1.] ] = 0.5

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_60 (Dense)            (None, 6)                 54        
                                                                 
 dense_61 (Dense)            (None, 8)                 56        
                                                                 
 dense_62 (Dense)            (None, 10)                90        
                                                                 
 dense_63 (Dense)            (None, 8)                 88        
                                                                 
 dense_64 (Dense)            (None, 2)                 18        
                                                                 
Total params: 306
Trainable params: 306
Non-trainable params: 0
_________________________________________________________________


In [386]:
model.fit(X_train,y_train,epochs=400,verbose=2,batch_size=32)

Epoch 1/400
25/25 - 1s - loss: 0.6843 - accuracy: 0.5113 - 675ms/epoch - 27ms/step
Epoch 2/400
25/25 - 0s - loss: 0.6805 - accuracy: 0.5775 - 46ms/epoch - 2ms/step
Epoch 3/400
25/25 - 0s - loss: 0.6764 - accuracy: 0.5925 - 44ms/epoch - 2ms/step
Epoch 4/400
25/25 - 0s - loss: 0.6719 - accuracy: 0.6313 - 47ms/epoch - 2ms/step
Epoch 5/400
25/25 - 0s - loss: 0.6663 - accuracy: 0.6575 - 61ms/epoch - 2ms/step
Epoch 6/400
25/25 - 0s - loss: 0.6593 - accuracy: 0.6812 - 86ms/epoch - 3ms/step
Epoch 7/400
25/25 - 0s - loss: 0.6509 - accuracy: 0.6825 - 61ms/epoch - 2ms/step
Epoch 8/400
25/25 - 0s - loss: 0.6410 - accuracy: 0.6900 - 59ms/epoch - 2ms/step
Epoch 9/400
25/25 - 0s - loss: 0.6300 - accuracy: 0.7050 - 69ms/epoch - 3ms/step
Epoch 10/400
25/25 - 0s - loss: 0.6180 - accuracy: 0.7138 - 76ms/epoch - 3ms/step
Epoch 11/400
25/25 - 0s - loss: 0.6071 - accuracy: 0.7163 - 62ms/epoch - 2ms/step
Epoch 12/400
25/25 - 0s - loss: 0.5947 - accuracy: 0.7287 - 37ms/epoch - 1ms/step
Epoch 13/400
25/25 - 0s

In [387]:
y_pred = model.predict(X_test)

In [388]:
y_pred

array([[0.49239904, 0.3720749 ],
       [0.5489532 , 0.22440264],
       [0.9342698 , 0.04901204],
       [0.525213  , 0.34806338],
       [0.6640537 , 0.23897341],
       [0.74900925, 0.1327109 ],
       [0.79085624, 0.12126747],
       [0.1210736 , 0.6462748 ],
       [0.21667624, 0.47739193],
       [0.18240175, 0.5629625 ],
       [0.38929683, 0.38704184],
       [0.12395996, 0.62016076],
       [0.80966437, 0.15129474],
       [0.19280744, 0.6047381 ],
       [0.9769672 , 0.01485059],
       [0.24907866, 0.5659949 ],
       [0.255264  , 0.47918057],
       [0.11621547, 0.64496106],
       [0.2789913 , 0.48632798],
       [0.07535836, 0.6963316 ],
       [0.31294602, 0.47193167],
       [0.10963002, 0.647482  ],
       [0.23152325, 0.52699125],
       [0.85743916, 0.08656445],
       [0.33941787, 0.4769733 ],
       [0.6380192 , 0.22673118],
       [0.26638854, 0.5520957 ],
       [0.97021633, 0.01943269],
       [0.9013232 , 0.07820266],
       [0.30374202, 0.45499104],
       [0.

In [389]:
y_test

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.

In [390]:
y_test = np.argmax(y_test,axis = 1)
y_predict = np.argmax(y_pred,axis=1)

In [391]:
y_test.shape

(200,)

In [392]:
y_predict

array([0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1,
       0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0,
       1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0,
       0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1,
       1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1,
       0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0,
       0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0,
       1, 0])

In [393]:
confumat = metrics.confusion_matrix(y_test,y_predict)

In [394]:
confumat

array([[81, 19],
       [18, 82]])

In [395]:
result = metrics.classification_report(y_test,y_predict)
print()
print('Classification Results')
print(result)


Classification Results
              precision    recall  f1-score   support

           0       0.82      0.81      0.81       100
           1       0.81      0.82      0.82       100

    accuracy                           0.81       200
   macro avg       0.82      0.81      0.81       200
weighted avg       0.82      0.81      0.81       200

